In [ ]:
import os
import sys
import torch.nn.functional as F
from io import BytesIO
from pathlib import Path
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from PIL import Image
from dataclasses import dataclass
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import torchmetrics
from torchvision import transforms
import os
from sklearn.model_selection import train_test_split
from torchinfo import summary
from torchmetrics import Accuracy
from transformers import AutoImageProcessor,AutoModel
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import pickle
import io
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

In [ ]:
dataset = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
train_data = '/kaggle/input/isic-2024-challenge/test-metadata.csv'

In [ ]:
df_train=pd.read_csv(train_data)
train_drop=['sex','anatom_site_general',
            'patient_id','image_type','tbp_tile_type','tbp_lv_location',
            'attribution','tbp_lv_location_simple',
            'copyright_license']
df_train.drop(train_drop,axis=1,inplace=True)
temp=df_train['age_approx'].median()
df_train['age_approx']=df_train['age_approx'].fillna(temp)
df_train.dropna(axis=1,inplace=True)

In [ ]:
class HDF5Dataset(Dataset): 
    def __init__(self, data, metadata, transform=None):
        self.data = h5py.File(data, 'r')
        if type(metadata) is pd.DataFrame:
            self.metadata = metadata
        else:
            self.metadata = pd.read_csv(metadata)
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx): 
        img_name = self.metadata['isic_id'].iloc[idx]  
        image = np.array(self.data[img_name])
        image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)/255 

        if self.transform:
            augmented = self.transform(image=image)  
            image = augmented['image']  
        meta=torch.from_numpy(np.asarray(self.metadata.iloc[idx])[1:].astype('f'))
        return image,meta    


train_transform = A.Compose([
    A.Resize(height=384, width=384), 
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

train_dataset = HDF5Dataset(dataset, df_train, transform=train_transform) 
train_load = DataLoader(train_dataset,
                        batch_size=32,
                        shuffle=False,
                        num_workers=4
                       )

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class model_4(nn.Module):
    def __init__(self,model):
        super().__init__()
        self.model=model
        self.relu=nn.ReLU()
        self.linear1=nn.Linear(1024,1)
        self.sigmoid=nn.Sigmoid()
        self.linear2=nn.Linear(34,1024)
        self.bn1=nn.BatchNorm1d(1024)
        self.rel1=nn.ReLU()
        self.drop=nn.Dropout(0.3)
        self.linear3=nn.Linear(1024,128)
        self.bn2=nn.BatchNorm1d(128)
        self.rel2=nn.ReLU()
        self.drop1=nn.Dropout(0.5)
        self.linear4=nn.Linear(1024+128,1)
        
        self.linear5=nn.Linear(128,1)
        self.linear6=nn.Linear(1024,1)
    def forward(self,x,meta):
        x=self.model(x)[1]
        x1=self.sigmoid(self.linear6(x))
        meta=self.rel2(self.bn2(self.linear3(self.drop(self.rel1(self.bn1(self.linear2(meta)))))))
        x2=self.sigmoid(self.linear5(meta))
        x=torch.cat([x,meta],axis=1)
        x=self.sigmoid(self.linear4(self.drop1(x)))
        return x,x1,x2

In [ ]:
model = pickle.load(open('/kaggle/input/num-1-sol/num_1_sol.sav', 'rb'))
# model.load_state_dict(torch.load(f="/kaggle/input/num-1-sol/model_ezy6.pth"))

In [ ]:
fi_hdf5=h5py.File('/kaggle/input/isic-2024-challenge/test-image.hdf5')
df_train=pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
isic_id=df_train['isic_id'].to_numpy()
isic_id=isic_id.astype('str')

In [ ]:
preds=[]
model.to(device)
model.eval()
with torch.inference_mode():
    for image,meta in train_load:
        x=image.to(device)
        meta=meta.to(device)
        pred,pred1,pred2=model(x,meta)
        preds.append(pred.cpu())
        del x
        del pred
        del pred1
        del pred2
        del meta
        torch.cuda.empty_cache()

In [ ]:
predictions=[]
for pred in preds:
    for pre in pred.tolist():
        predictions.append(pre)

In [ ]:
preds=np.array(predictions)
preds=preds.squeeze()
preds[np.isnan(preds)]=0
preds= pd.DataFrame(preds)
df_sub=pd.concat([df_train['isic_id'],preds],axis=1)
df_sub

In [ ]:
df_sub.to_csv("submission.csv", index=False)